## Assignment 2: Part 1 (ML) [7 pts]

<span style="color:blue">
    
### In sections 1.2-1.5 of the Machine Learning notebook, there are tasks for you to complete. Be sure to submit BOTH the Machine Learning demo notebook and this notebook.
</spaN>

## Assignment 2: Part 2 (NLP) [8 pts]

### 2.1 Fast Text [3 pts]

FastText[1] is a neural network based text classification model designed to be computationally efficient. Your task is to implement the FastText algorithm by completeing the code in the following cells. You will need to read through the provided fastText.pdf paper, which explains the algorithm. You do not need to implement Hierarchical softmax (2.1) or N-gram features (2.2), you only need to implement the basic architecture described in (2). 

The FastText model will be trained using mini-batch gradient descent. When the training data are sequences of variable lengths we can not simply stack multiple training sequences into one tensor. Instead, it is common to assume that there is a maximal sequence length, so that all sequences in a batch are fitted into tensors of the same dimensions. For sequences shorter than the maximal length, we append them with a special pad word so that all sequences in a batch are of the same length. A *pad word* is a special token, whose embedding is an all-zero vector, so that the presence of pad words does not change the output of the model. In this code, the pad word has an ID of 0, when implementing your embeddings you should ensure that this ID is always embedded to a vector of all zeros. Additionally, you will need to know how many words are in each input sentence (before they got padded to be the same length), this is provided as an input parameter to your FastText model.

[1] Joulin, Armand and Grave, Edouard and Bojanowski, Piotr and Mikolov, Tomas. Bag of Tricks for Efficient Text Classification. arXiv preprint arXiv:1607.01759., 2016. [INCLUDED AS PART OF ASSIGNMENT 2 .ZIP PACKAGE]



In [1]:
# coding: utf-8
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np

import collections
import math
import os
import random

import nltk
nltk.download('punkt')
from nltk import word_tokenize
from collections import namedtuple

import sys, getopt

from random import shuffle


num_classes = 3

learning_rate = 0.005
num_epochs = 3
batch_size = 3
embedding_dim = 3

[nltk_data] Downloading package punkt to C:\Users\yadu
[nltk_data]     k\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


<span style="color:blue">
    
### You need to complete the foward() and __init__() functions below [3 pts]
</span>

In [266]:
class FastText(nn.Module):
    """Define the computation graph for fasttext model."""
    
    def __init__(self, vocab_size, num_classes, embedding_dim, learning_rate):
        """Init the model with default parameters/hyperparameters."""
        super(FastText, self).__init__()
        self.num_classes = num_classes
        self.embedding_dim = embedding_dim
        self.learning_rate = learning_rate
        self.loss_func = F.cross_entropy
        # TODO: create all the variables (weights) that the model needs here
        # raise NotImplementedError
        
        self.W = nn.Linear(batch_size,num_classes)
        self.embedder = nn.Embedding(vocab_size, embedding_dim)
        
        self.optimizer = torch.optim.SGD(self.parameters(), lr=learning_rate)
    
    def forward(self, x, sens_lengths):
        # TODO: implement the FastText computation
        
        x=self.embedder(x);
        x=self.W(x.sum(dim = 1)/sens_lengths)

        
        return x

In [267]:
from fasttext import load_question_2_1, train_fast_text

word_to_id, train_data, valid_data, test_data = load_question_2_1('question_2-1_data')
model = FastText(len(word_to_id)+2, num_classes, embedding_dim=embedding_dim, learning_rate=learning_rate)

model_file_path = os.path.join('models', 'fasttext_model_file_q2-1')
train_fast_text(model, train_data, valid_data, test_data, model_file_path, batch_size=batch_size, num_epochs=num_epochs)

number of sequences is 8216. 
PAD word id is 0 .
Unknown word id is 1 .
size of vocabulary is 3666. 
read 1000 sentences from question_2-1_data\sentences_train.txt .
read 500 sentences from question_2-1_data\sentences_dev.txt .
read 500 sentences from question_2-1_data\sentences_test.txt .
Epoch 0 : train loss = 1.177636589910891 , validation accuracy = 0.3619999885559082 .
Epoch 1 : train loss = 1.1040675885326512 , validation accuracy = 0.3779999911785126 .
Epoch 2 : train loss = 1.108659344392496 , validation accuracy = 0.3840000033378601 .
Accuracy on the test set : 0.3499999940395355.


### 2.2 Question Classification [3 pts]

Understanding questions is a key problem in chatbots and question answering systems. In the open-domain setting, it is difficult to find right answers in the huge search space. To tackle the problem, one approach is to categorize questions into a finite set of semantic classes, and each semantic class corresponds to a small answer space.

<span style="color:blue">
    
### Your task is to implement a question classification model in Pytorch, and apply it to the question_2_2_data provided in this assignment.
</span>

Notes: 


-  Please do NOT submit your data directories, pretrained word embeddings, and Pytorch library!

-  You may consider reusing parts of the code above

-  Code must be submitted with the assignment for purposes of plagiarism detection

### Dataset

The dataset provided contains three files: **train.json**, **validation.json**, and **test.json**, which are the training dataset, validation dataset, and the test dataset, respectively. 
See an example below: 
```
{
   "ID": S1,
   "Label": 3,
   "Sentence":"What country has the best defensive position in the board game Diplomacy ?"
}
```
In the training set and the validation set, the response variable is called `Label`. Your task is to predict the `Label` for each sentence in the test set. 

### Evaluation

The performance of your prediction will be evaluated automatically on Kaggle using __Accuracy__ , which is defined as the number of correct predictions divided by the total number of sentences in the test set (https://classeval.wordpress.com/introduction/basic-evaluation-measures/).

It is important to understand that the leaderboard score will be only computed based on the half of the test cases, and the remaining half will be computed after the deadline based on your selected submission. This process will ensure that your performance is not only applicable for the known test cases, but also generalised to the unknown test cases. We will combine these two performances to score the first assignment.

Your score will be computed using a lower bound and an upper bound, which will be shown on the Kaggle leader board. 
Achieving an accuracy equal and below the lower bound amounts to a grade of zero, while achieving the upper bound amounts to the full points (here 3 points, see score distribution here below).
Consequently, your score for this competition task will be calculated based on:

$$
    \operatorname{Your\_Score} = \frac{Your\_Accuracy - Lower\_Bound}{Upper\_Bound - Lower\_Bound} * 3
$$
Notes about the lower bound and upper bounds predictors:

* The **lower bound** is the performance obtained by a classifer that always picks the majority class according to the class distribution in the training set.
* The **upper bound** is generated by an "in-house" classifier trained on the same dataset that you were given.

There are many possibilities to achieve better results than this. However, the **only** labeled training dataset to train your model should be the provided **train.json**. 
If you obtain a better performance than the upper bound, then you will have a grade higher than 3 points for this question. This can be useful to compensate for any lost points for the whole assignment.
However, the total mark of this assignment is capped at 10 marks.

### Kaggle competition

- You will be given a link to join the competition during your labs.
- Before submitting the result, first go to **team** menu and change your **team name** as **your university id**.
- You need to upload the generated result file to Kaggle. The result file should be in the following format
```
id,category
S101,0
S201,1
S102,2
...
```
- Note that you are only allowed to upload **5 copies** of your results to Kaggle per day. Make every upload count, and don't waste your opportunities!

**NB** you need to fill in the cells below with your code. If you fail to provide the code, you will get zero for this question. Your code should be well documented and provide methods to generate the prediction files and compute accuracy on the validation set.

In [268]:
import json # You can use this library to read the .json files into a Python dict: https://docs.python.org/2/library/json.html
from nltk import word_tokenize # You can use this to tokenize strings, or do your own pre-processing.

In [337]:
"""
Your tasks are to
    1. Read in the .json files and create Dataset objects from them. The dataset constructor requires two parameters: a list of
        sentences (where each sentence is a list of word ids) and a list of labels (or None is there are no labels).
        You will need to apply appropriate preprocessing to the raw text to get in the appropriate form.
    2. Run the train_fast_text() function on these Datasets and your model.
    3. Convert the output file of predictions into the correct format for Kaggle. 
        Kaggle expects a csv with two columns, id and category. You need to have these two column headers as the first row.
        Your csv should not include any whitespace.
    4. Change the model hyper parameters, training settings, text preprocessing, or anything else you see fit
        in order to improve your models performance.
"""

num_classes = 6
pad_word_id = 0
unknown_word_id = 1
batch_size = 4
embedding_dim = 4

from prepros import preprocessor
from fasttext import Dataset
from fasttext import map_token_seq_to_word_id_seq
import pandas as pd

train = pd.read_json('question_2-2_data/train.json') # Read the training dataset
val = pd.read_json('question_2-2_data/validation.json') # Read the validation dataset
test = pd.read_json('question_2-2_data/test.json') # Read the testing dataset

def read_labeled_dataset(Dataset, df, word_to_id, is_label):
    """ Read labeled dataset.

        Args:
            df (data frame) : training, validation and testing dataset.
            word_to_id (dictionary) : a dictionary mapping words to their ids.
            is_label (boolean) : true if the dataset does not have labels
    """
    data = []
    for sens in df.Sentence:
        word_id_seq = map_token_seq_to_word_id_seq(word_tokenize(sens), word_to_id)
        if len(word_id_seq) > 0:
            data.append(word_id_seq)
    if is_label:
        labeled_set = Dataset(sentences=data)
    else:
        labeled_set = Dataset(sentences=data,labels=df.Label.tolist())

    return labeled_set

def load_question_2_2():
    # Function in fasttext.py modified and reused 
    word_to_id = build_vocab(train)
    train_dataset = read_labeled_dataset(Dataset, train, word_to_id,False)
    val_dataset = read_labeled_dataset(Dataset, val, word_to_id,False)
    test_dataset = read_labeled_dataset(Dataset, test, word_to_id,True)
    return word_to_id, train_dataset, val_dataset, test_dataset

def build_vocab(df):
    """ Build a vocabulary from a train set.

        Args:
            df (data frame) : the training dataset.
    """
    data = []
    for line in df.Sentence:
        tokens = word_tokenize(line)
        data.extend(tokens)
    print('number of sequences is %s. ' % len(data))
    count = [['$PAD$', pad_word_id], ['$UNK$', unknown_word_id]]
    sorted_counts = collections.Counter(data).most_common()
    count.extend(sorted_counts)
    word_to_id = dict()
    for word, _ in count:
        word_to_id[word] = len(word_to_id)

    print("PAD word id is %s ." % word_to_id['$PAD$'])
    print("Unknown word id is %s ." % word_to_id['$UNK$'])
    print('size of vocabulary is %s. ' % len(word_to_id))
    return word_to_id


word_to_id, train_data, valid_data, test_data = load_question_2_2()
model = FastText(len(word_to_id)+2, num_classes, embedding_dim=embedding_dim, learning_rate=0.05)

model_file_path = os.path.join('models', 'fasttext_model_file_q2-2')
train_fast_text(model, train_data, valid_data, test_data, model_file_path, batch_size=batch_size, num_epochs=400)

model_df = pd.read_csv("models/fasttext_model_file_q2-2predictions.csv",names = ['category'])
model_df['id']=test.ID
model_df=model_df[['id','category']]
model_df.to_csv("models/output.csv",index=False)


number of sequences is 76218. 
PAD word id is 0 .
Unknown word id is 1 .
size of vocabulary is 8908. 
[[4, 11, 3, 25, 6, 3, 3037, 33, 5, 179, 643, 3, 163, 47, 6, 3038, 2], [14, 185, 9, 31, 707, 9, 2], [4, 16, 3, 201, 66, 211, 3039, 23, 3, 404, 438, 2], [4, 5, 3, 79, 6, 3, 6164, 2340, 1705, 9, 3040, 5, 3, 113, 17, 125, 186, 330, 13, 107, 13, 3, 6165, 6, 114, 35, 9, 2], [4, 16, 3, 3041, 1123, 2], [10, 69, 2341, 218, 42, 950, 355, 43, 951, 2], [14, 821, 31, 2342, 3042, 2], [50, 708, 87, 3043, 32], [4, 404, 3044, 18, 1706, 212, 12, 331, 58, 3, 308, 6, 1124, 2], [4, 12, 3, 134, 211, 23, 521, 309, 522, 1707, 12, 822, 2]]
[[10, 21, 26, 53, 8, 1174, 579, 6, 125, 3, 54, 33, 16, 1085, 40, 3, 148, 25, 1, 2], [4, 5, 3, 66, 1290, 672, 7, 1148, 2], [4, 16, 3, 41, 187, 258, 6, 294, 7, 3, 34, 2], [4, 5, 8, 6367, 2], [10, 69, 1269, 1065, 218, 42, 2617, 533, 53, 951, 2], [10, 22, 1103, 1976, 1374, 7, 8, 1, 2], [24, 5, 3, 8313, 2], [14, 5, 8901, 8902, 2], [4, 481, 3, 1131, 13, 1383, 2], [48, 5, 1355, 165

C:\Users\yadu k\Anaconda3\lib\site-packages\torch\serialization.py:256: UserWarning: Couldn't retrieve source code for container of type FastText. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


Epoch 0 : train loss = 1.605854914984821 , validation accuracy = 0.37299999594688416 .
Epoch 1 : train loss = 1.5216203591557245 , validation accuracy = 0.33500000834465027 .
Epoch 2 : train loss = 1.468641428058821 , validation accuracy = 0.4690000116825104 .
Epoch 3 : train loss = 1.421107831354044 , validation accuracy = 0.44999998807907104 .
Epoch 4 : train loss = 1.380580399557548 , validation accuracy = 0.44999998807907104 .
Epoch 5 : train loss = 1.343000549971097 , validation accuracy = 0.45399999618530273 .
Epoch 6 : train loss = 1.3124449834083511 , validation accuracy = 0.44699999690055847 .
Epoch 7 : train loss = 1.278193418324698 , validation accuracy = 0.5189999938011169 .
Epoch 8 : train loss = 1.2526728123282387 , validation accuracy = 0.43799999356269836 .
Epoch 9 : train loss = 1.2271761584758887 , validation accuracy = 0.5410000085830688 .
Epoch 10 : train loss = 1.2032419078524208 , validation accuracy = 0.5709999799728394 .
Epoch 11 : train loss = 1.178447973871429

Epoch 185 : train loss = 0.3269473753977935 , validation accuracy = 0.8669999837875366 .
Epoch 186 : train loss = 0.2892742252756405 , validation accuracy = 0.8820000290870667 .
Epoch 187 : train loss = 0.30660701457448025 , validation accuracy = 0.7760000228881836 .
Epoch 188 : train loss = 0.31101143315832164 , validation accuracy = 0.8939999938011169 .
Epoch 189 : train loss = 0.31533068474404446 , validation accuracy = 0.8679999709129333 .
Epoch 190 : train loss = 0.3225491579981698 , validation accuracy = 0.8809999823570251 .
Epoch 191 : train loss = 0.2953428821111706 , validation accuracy = 0.8960000276565552 .
Epoch 192 : train loss = 0.2973221798836365 , validation accuracy = 0.7699999809265137 .
Epoch 193 : train loss = 0.31709824445994056 , validation accuracy = 0.878000020980835 .
Epoch 194 : train loss = 0.2904657211111473 , validation accuracy = 0.8230000138282776 .
Epoch 195 : train loss = 0.309436338362555 , validation accuracy = 0.8009999990463257 .
Epoch 196 : train l

Epoch 367 : train loss = 0.1041439555372174 , validation accuracy = 0.9010000228881836 .
Epoch 368 : train loss = 0.1364802984139327 , validation accuracy = 0.9160000085830688 .
Epoch 369 : train loss = 0.09690852230006276 , validation accuracy = 0.9350000023841858 .
Epoch 370 : train loss = 0.09512444365919255 , validation accuracy = 0.9369999766349792 .
Epoch 371 : train loss = 0.1359186014222246 , validation accuracy = 0.9350000023841858 .
Epoch 372 : train loss = 0.16253207960429297 , validation accuracy = 0.9139999747276306 .
Epoch 373 : train loss = 0.1264662301208341 , validation accuracy = 0.9369999766349792 .
Epoch 374 : train loss = 0.11603830921068319 , validation accuracy = 0.9350000023841858 .
Epoch 375 : train loss = 0.0982785168063501 , validation accuracy = 0.9390000104904175 .
Epoch 376 : train loss = 0.13879805430824346 , validation accuracy = 0.9359999895095825 .
Epoch 377 : train loss = 0.0937261870517776 , validation accuracy = 0.9129999876022339 .
Epoch 378 : trai

### 2.3 Comparison between Absolute Discounting and Kneser Ney smoothing [2pts]

Read the code below for interpolated absolute discounting and implement Kneser Ney smoothing in Python. It is sufficient to assume that the highest order of ngram is two and the discount is 0.75. Evaluate your program on the following ngram corpus and compute the distribution $p(x | \text{Granny})$ for all possible unigrams in the given corpus. 

<span style="color:blue">
    
### Explain what make the differences regarding the prediction results between interpolated absolute discounting and Kneser Ney smoothing.
</span>



In [321]:
ngram_corpus = ['Sam eats apple',
          "Granny plays with Sam",
           "Sam plays with Smith",
           "Sam likes Smith",
          "Sam likes apple",
                "Sam likes sport",
                "Sam plays tennis",
                "Sam likes games",
                "Sam plays games",
          "Sam likes apple Granny Smith"]

from nltk import word_tokenize
from nltk.util import ngrams
from collections import Counter

class NgramStats:
    """ Collect unigram and bigram statistics. """
    
    def __init__(self):
        self.bigram_to_count = Counter([])
        self.unigram_to_count = dict()
        
    def collect_ngram_counts(self, corpus):
        """Collect unigram and bigram counts from the given corpus."""
        unigram_counter = Counter([])
        for sentence in corpus:
            tokens = word_tokenize(sentence)
            bigrams = ngrams(tokens, 2)
            unigrams = ngrams(tokens, 1)
            self.bigram_to_count += Counter(bigrams)
            unigram_counter += Counter(unigrams)
        self.unigram_to_count = {k[0]:int(v) for k,v in unigram_counter.items()}

In [322]:
stats = NgramStats()         
stats.collect_ngram_counts(ngram_corpus)
print(stats.bigram_to_count)
print(stats.unigram_to_count)

Counter({('Sam', 'likes'): 5, ('Sam', 'plays'): 3, ('plays', 'with'): 2, ('likes', 'apple'): 2, ('Sam', 'eats'): 1, ('eats', 'apple'): 1, ('Granny', 'plays'): 1, ('with', 'Sam'): 1, ('with', 'Smith'): 1, ('likes', 'Smith'): 1, ('likes', 'sport'): 1, ('plays', 'tennis'): 1, ('likes', 'games'): 1, ('plays', 'games'): 1, ('apple', 'Granny'): 1, ('Granny', 'Smith'): 1})
{'Sam': 10, 'eats': 1, 'apple': 3, 'Granny': 2, 'plays': 4, 'with': 2, 'Smith': 3, 'likes': 5, 'sport': 1, 'tennis': 1, 'games': 2}


In [323]:
# Interpolated Absolute Discounting
import operator
    
class AbsDist:
    """
     Implementation of Interpolated Absolute Discounting
     
     Reference: slide 25 in https://nlp.stanford.edu/~wcmac/papers/20050421-smoothing-tutorial.pdf
    """
    def __init__(self, ngram_stats):
        """ Initialization
        
            Args:
                ngram_stats (NgramStats) : ngram statistics.
        """
        self.unigram_freq = float(sum(ngram_stats.unigram_to_count.values()))
        self.stats= ngram_stats
    
    def compute_prop(self, bigram, discount = 0.75):
        """ Compute probability p(y | x)
        
            Args:
                bigram (string tuple) : a bigram (x, y), where x and y denotes an unigram respectively.
                discount (float) : the discounter factor for the linear interpolation.
        """
        preceding_word_count = 0
        if bigram[0] in self.stats.unigram_to_count:
            preceding_word_count = self.stats.unigram_to_count[bigram[0]]
            
        if preceding_word_count > 0:
            left_term = 0
            if bigram in self.stats.bigram_to_count:
                bigram_count = float(self.stats.bigram_to_count[bigram])
                left_term = (bigram_count - discount)/preceding_word_count
            right_term = 0
            if bigram[1] in self.stats.unigram_to_count:
                current_word_count = self.stats.unigram_to_count[bigram[1]]
                num_bigram_preceding_word = 0
                for c_bigram in self.stats.bigram_to_count.keys():
                    if c_bigram[0] == bigram[0] :
                        num_bigram_preceding_word += 1
                normalization_param = (discount * num_bigram_preceding_word)/ preceding_word_count 
                p_unigram = current_word_count/self.unigram_freq
                right_term = normalization_param * p_unigram
            return left_term + right_term
        
        return 0

In [324]:
def compute_prop_abs_dist(ngram_stats, preceding_unigram, d = 0.75):
    """ Compute the distribution p(y | x) of all y given preceding_unigram

        Args:
            preceding_unigram (string) : the preceding unigram.
            d (float) : the discounter factor for the linear interpolation.
    """
    absDist_1 = AbsDist(ngram_stats)
    c_unigram_to_prob = dict()
    for c_unigram in ngram_stats.unigram_to_count.keys():
        if not c_unigram in c_unigram_to_prob:
            c_unigram_to_prob[c_unigram] = absDist_1.compute_prop((preceding_unigram, c_unigram), d)
  
    sorted_prob = sorted(c_unigram_to_prob.items(), key=operator.itemgetter(1))
    return sorted_prob


print(compute_prop_abs_dist(stats, 'Granny'))

[('eats', 0.022058823529411763), ('sport', 0.022058823529411763), ('tennis', 0.022058823529411763), ('Granny', 0.044117647058823525), ('with', 0.044117647058823525), ('games', 0.044117647058823525), ('apple', 0.0661764705882353), ('likes', 0.11029411764705882), ('Smith', 0.19117647058823528), ('plays', 0.21323529411764705), ('Sam', 0.22058823529411764)]


In [335]:
class AbsDist_1:
    def __init__(self, ngram_stats):
        self.unigram_freq = float(sum(ngram_stats.unigram_to_count.values()))
        self.stats= ngram_stats   
    def compute_prop_1(self, bigram, discount = 0.75):
        preceding_word_count = 0
        if bigram[0] in self.stats.unigram_to_count:
            preceding_word_count = self.stats.unigram_to_count[bigram[0]]
            
        if preceding_word_count > 0:
            left_term = 0
            if bigram in self.stats.bigram_to_count:
                bigram_count = float(self.stats.bigram_to_count[bigram])
                left_term = (bigram_count - discount)/preceding_word_count
            right_term = 0
            if bigram[1] in self.stats.unigram_to_count:
                #current_word_count = self.stats.unigram_to_count[bigram[1]]
                num_bigram_preceding_word = 0
                for c_bigram in self.stats.bigram_to_count.keys():
                    if c_bigram[0] == bigram[0] :
                        num_bigram_preceding_word += 1
                normalization_param = (discount * num_bigram_preceding_word)/ preceding_word_count
                #print(self.stats.bigram_to_count)
                
                bigram_count = 0
                for key in self.stats.bigram_to_count:
                    bigram_count += 1
                
                p_cont = num_bigram_preceding_word/bigram_count
                right_term = normalization_param * p_cont
            return left_term + right_term
        
        return 0

def compute_prop_KN(ngram_stats, preceding_word, d=0.75):
    # Implement Kneser Ney Smoothing here.
    # Hint: try to reuse the above code as much as possible.
    
    absDist_1 = AbsDist_1(ngram_stats)
    c_unigram_to_prob = dict()
    for c_unigram in ngram_stats.unigram_to_count.keys():
        if not c_unigram in c_unigram_to_prob:
            c_unigram_to_prob[c_unigram] = absDist_1.compute_prop_1((preceding_word, c_unigram), d)
  
    sorted_prob = sorted(c_unigram_to_prob.items(), key=operator.itemgetter(1))
    return sorted_prob
    
    

print(compute_prop_KN(stats, 'Granny'))

[('Sam', 0.09375), ('eats', 0.09375), ('apple', 0.09375), ('Granny', 0.09375), ('with', 0.09375), ('likes', 0.09375), ('sport', 0.09375), ('tennis', 0.09375), ('games', 0.09375), ('plays', 0.21875), ('Smith', 0.21875)]


<span style="color:blue">
    
EXPLAIN THE DIFFERENCES REGARDING PREDICTION RESULTS HERE
</span>
